### Imports necessary for running the program

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun May  5 23:01:42 2019

@author: Ahmed
"""

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
import pandas as pd
'''df = pd.read_excel("MLDataSet.xlsx")
df2 = pd.read_excel("ADRs.xlsx")
df3 = pd.read_excel("DS.xlsx")
df4 = pd.read_excel("Mental.xlsx")

df5=[df, df2,  df3, df4]

result = pd.concat([df,df2,df3, df4], axis=1, join_axes=[df.index])'''
#result.to_excel('result.xlsx')


'df = pd.read_excel("MLDataSet.xlsx")\ndf2 = pd.read_excel("ADRs.xlsx")\ndf3 = pd.read_excel("DS.xlsx")\ndf4 = pd.read_excel("Mental.xlsx")\n\ndf5=[df, df2,  df3, df4]\n\nresult = pd.concat([df,df2,df3, df4], axis=1, join_axes=[df.index])'

In [2]:
def removeColumns(result):
    del result['Pain']
    del result['Content']
    del result['Filtered']
    del result['Stemmed']
    del result['big1']
    del result['big2']
    del result['small1']
    del result['small2']
    del result['Height']
    del result['Joined']
    del result['Posted']
    

In [188]:
def vectorizeCount(result):
    '''for i in range(len(result)):
        if result[i]==0:
            result[i]=0
        if result[i]>0 and result[i]<=3:
            result[i]=1
        if result[i]>3:
            result[i]=2'''
    for i in range(len(result)):
        if result.at[i,'MentalCount']==0:
            result.at[i,'MentalCount']=0
        if result.at[i,'MentalCount']>0 and result.at[i,'MentalCount']<=3:
            result.at[i,'MentalCount']=1
        if result.at[i,'MentalCount']>3:
            result.at[i,'MentalCount']=2
            
        if result.at[i,'ADRCount']==0:
            result.at[i,'ADRCount']=0
        if result.at[i,'ADRCount']>0 and result.at[i,'MentalCount']<=3:
            result.at[i,'ADRCount']=1
        if result.at[i,'ADRCount']>3:
            result.at[i,'ADRCount']=2

        if result.at[i,'DieaseCount']==0:
            result.at[i,'DieaseCount']=0
        if result.at[i,'DieaseCount']>0 and result.at[i,'DieaseCount']<=3:
            result.at[i,'DieaseCount']=1
        if result.at[i,'DieaseCount']>3:
            result.at[i,'DieaseCount']=2
    return result

### manipulating data to prepare for classification

In [56]:
def manipulate(result,labelfile,label):
    from collections import Counter
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    result['Gender']=imp_mean.fit_transform(result['Gender'].values.reshape(-1, 1))
    for i in range(len(result)):
        if result.at[i,'Gender']=='Male':
            result.at[i,'Gender']=0
        if result.at[i,'Gender']=='Female':
            result.at[i,'Gender']=1
    
    labels=labelfile.iloc[:][label]
    print(Counter(labels))
    labels=LabelEncoder().fit_transform(labels)
    if 'Unnamed: 0' in result:
        del result['Unnamed: 0'] 
    #removeColumns(result)
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')
    result['Age']=imp_mean.fit_transform(result['Age'].values.reshape(-1, 1))
    if 'Height' in result:
        result['Height']=imp_mean.fit_transform(result['Height'].values.reshape(-1, 1))
    result=vectorizeCount(result)
    return result,labels

In [8]:

#asklabel=pd.read_excel('asklabels.xlsx')
#labels=asklabel.iloc[:]['MICROCEPHALY , EPILEPSY , AND DIABETES SYNDROME']

In [63]:

def SMOTE(begin,columns):
    from imblearn.over_sampling import SMOTENC
    x=[1]
    x.extend(list(range(begin,len(columns))))
    sm=SMOTENC(random_state=42, categorical_features=x)
    return sm


### Datasets

In [53]:
label='ADRCount'
asklabel='ADRCount'

#from collections import Counter
#X_res, y_res = sm.fit_resample(result, result_labels)
#print(Counter(result_labels))
#print(Counter(y_res))
def prepareDataset(label,asklab):
    from imblearn.over_sampling import SMOTENC
    
    
    
    result=pd.read_excel('result_reduced.xlsx')
    weighted=pd.read_excel('Weighted_reduced.xlsx')
    blood=pd.read_excel('blood_reduced.xlsx')
    ask=pd.read_excel('askapatient_reduced.xlsx')

    resultlabel=pd.read_excel('resultlabels.xlsx')
    weightedlabel=pd.read_excel('Weightedlabels.xlsx')
    bloodlabel=pd.read_excel('bloodlabels.xlsx')
    asklabel=pd.read_excel('asklabels.xlsx')

    result,result_labels=manipulate(result,resultlabel,label)
    weighted,weighted_labels=manipulate(weighted,weightedlabel,label)
    blood,blood_labels=manipulate(blood,bloodlabel,label)
    ask,ask_labels=manipulate(ask,asklabel,asklab)
    del result['weights2']
    del result['Height']
    
    sm1 = SMOTE(2,result.columns)
    sm2 = SMOTE(4,weighted.columns)
    sm3 = SMOTE(4,blood.columns)
    sm4 = SMOTE(2,ask.columns)
    #Applying SMOTENC
    result,result_labels=sm1.fit_resample(result,result_labels)
    weighted,weighted_labels=sm2.fit_resample(weighted,weighted_labels)
    blood,blood_labels=sm3.fit_resample(blood,blood_labels)
    ask,ask_labels=sm4.fit_resample(ask,ask_labels)
    return result,result_labels,weighted,weighted_labels,blood,blood_labels,ask,ask_labels


In [64]:
label='ADRCount'
asklabel='ADRCount'

#from collections import Counter
#X_res, y_res = sm.fit_resample(result, result_labels)
#print(Counter(result_labels))
#print(Counter(y_res))
def prepareDatasetNoSmote(label,asklab):
    
    result=pd.read_excel('result_reduced.xlsx')
    weighted=pd.read_excel('Weighted_reduced.xlsx')
    blood=pd.read_excel('blood_reduced.xlsx')
    ask=pd.read_excel('askapatient_reduced.xlsx')

    resultlabel=pd.read_excel('resultlabels.xlsx')
    weightedlabel=pd.read_excel('Weightedlabels.xlsx')
    bloodlabel=pd.read_excel('bloodlabels.xlsx')
    asklabel=pd.read_excel('asklabels.xlsx')

    result,result_labels=manipulate(result,resultlabel,label)
    weighted,weighted_labels=manipulate(weighted,weightedlabel,label)
    blood,blood_labels=manipulate(blood,bloodlabel,label)
    ask,ask_labels=manipulate(ask,asklabel,asklab)
    if 'Count' in label:
        result_labels = vectorizeCount(result_labels)
        weighted_labels = vectorizeCount(weighted_labels)
        blood_labels=vectorizeCount(blood_labels)
    if 'Count' in asklab:
        ask_labels = vectorizeCount(ask_labels)
    del result['weights2']
    del result['Height']
    
    
    #Applying SMOTENC
    #result,result_labels=sm.fit_resample(result,result_labels)
    #weighted,weighted_labels=sm.fit_resample(weighted,weighted_labels)
    #blood,blood_labels=sm.fit_resample(blood,blood_labels)
    #ask,ask_labels=sm.fit_resample(ask,ask_labels)
    return result,result_labels,weighted,weighted_labels,blood,blood_labels,ask,ask_labels


In [7]:
'''for i in weighted:
    if i not in result:
        if i != 'weights2':
            del weighted[i]
weighted.to_excel('Weighted.xlsx')'''

"for i in weighted:\n    if i not in result:\n        if i != 'weights2':\n            del weighted[i]\nweighted.to_excel('Weighted.xlsx')"

In [8]:
#ask_labels

In [9]:
'''for i in weighted:
    print (weighted[i])'''

'for i in weighted:\n    print (weighted[i])'

In [10]:
def get_score(model, X_train,X_test,Y_train,Y_test):
    model.fit(X_train,Y_train)
    return model.score(X_test,Y_test)

### 10-Fold Cross validation

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
#cross_val_score(RandomForestClassifier(n_estimators=1000),result,result_labels,cv=10)

In [12]:
#This was used for some other no longer existing purposes
def getMissingPercentage(feature):
    #nancount = int(result[result[feature].isnull()][feature].shape[0])
    nancount = int(result[result[feature]==1][feature].shape[0])
    size=int(result.shape[0])
    print (nancount)
    print ((nancount*100)/size)
#getMissingPercentage('Pvc')

In [ ]:
def importance(result,result_labels):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.feature_selection import SelectFromModel
    from collections import Counter
    
    X_train, X_test, Y_train, Y_test = train_test_split(result, result_labels, test_size=0.30, random_state=42)
    X_train, X_test, Y_train, Y_test = pd.DataFrame(X_train),pd.DataFrame(X_test),pd.DataFrame(Y_train),pd.DataFrame(Y_test),
    #print(Counter(result_labels))
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 500))
    sel.fit(X_train, Y_train)
    #print(sel.get_support())
    selected_feat= X_train.columns[(sel.get_support())]
    #print(len(selected_feat))
    #print(selected_feat)


    return selected_feat

In [173]:
label='Drug'
asklabel='Drug'

result,result_labels,weighted,weighted_labels,blood,blood_labels,ask,ask_labels=prepareDataset(label,asklabel)
res_feat=importance(result,result_labels)
wei_feat=importance(weighted,weighted_labels)
blood_feat=importance(blood,blood_labels)
ask_feat=importance(ask,ask_labels)



Counter({'Atenolol': 620, 'Lisinopril': 428, 'Amlodipine': 161, 'Hydrochlorothiazide': 129, 'Nadolol': 119, 'Diltiazem': 100})
Counter({'Lisinopril': 51, 'Atenolol': 45, 'Amlodipine': 14, 'Hydrochlorothiazide': 13})
Counter({'Atenolol': 186, 'Lisinopril': 134, 'Amlodipine': 63, 'Hydrochlorothiazide': 51, 'Diltiazem': 26})
Counter({'Lisinopril': 622, 'Atenolol': 134})


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when 

In [182]:
pd.DataFrame(weighted)[wei_feat]

,0,2,3,4,5,6,15
0,45,68,173,0,1,2,1
1,32,54,170,2,0,2,1
2,26,56,165,0,0,2,1
3,34,65,175,4,0,1,1
4,37,95,175,0,0,1,1
5,72,65,165,4,0,1,1
6,24,151,182,0,0,1,1
7,25,113,180,4,1,1,1
8,25,76,177,4,0,2,1
9,21,99,190,0,0,1,1


In [13]:
'''from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
kf = KFold(n_splits=10)

scores_rf=[]

for train_index, test_index in kf.split(result):
    X_train, X_test, Y_train, Y_test = result.iloc[train_index], result.iloc[test_index], result_labels.iloc[train_index],result_labels.iloc[test_index]
    scores_rf.append(get_score(RandomForestClassifier(n_estimators=400),X_train, X_test, Y_train, Y_test))'''

'from sklearn.model_selection import KFold\nfrom sklearn.ensemble import RandomForestClassifier\nkf = KFold(n_splits=10)\n\nscores_rf=[]\n\nfor train_index, test_index in kf.split(result):\n    X_train, X_test, Y_train, Y_test = result.iloc[train_index], result.iloc[test_index], result_labels.iloc[train_index],result_labels.iloc[test_index]\n    scores_rf.append(get_score(RandomForestClassifier(n_estimators=400),X_train, X_test, Y_train, Y_test))'

### Random Forest Classifier Function 

In [143]:
def randomForest(result,result_labels,estimators,DataSet):
    from sklearn.model_selection import KFold
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report
    from sklearn.metrics import f1_score
    from collections import Counter
    
    X_train, X_test, Y_train, Y_test = train_test_split(result, result_labels, test_size=0.30, random_state=42)
    print(Counter(result_labels))
    clf = RandomForestClassifier(n_estimators=estimators,random_state=0)
    clf.fit(X_train, Y_train)
    pred=clf.predict(X_test)
    accuracy=(accuracy_score(Y_test, pred))
    #print (accuracy)
    scores_rf=(precision_recall_fscore_support(Y_test,pred, average='macro'))
    #print(DataSet)
    #print (pred)
    #print(classification_report(Y_test,pred,labels=[0,1,2]))
        
    return accuracy,scores_rf

### Random Forest

In [197]:
label='Drug'
asklabel='Drug'
res_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
wei_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
bld_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
ask_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
for estimators in range (100,1000,100):
    print (estimators)
    result,result_labels,weighted,weighted_labels,blood,blood_labels,ask,ask_labels=prepareDataset(label,asklabel)
    res_acc,res_scores=randomForest(result,result_labels,estimators,"Result")
    wei_acc,wei_scores=randomForest(weighted,weighted_labels,estimators,"Weighted")
    blood_acc,blood_scores=randomForest(blood,blood_labels,estimators,"Blood")
    ask_acc,ask_scores=randomForest(ask,ask_labels,estimators,"Ask")
    print("Res: Accuracy: " + str(res_acc)+" # "+ "Scores: "+str(res_scores))
    print("Wei: Accuracy: " + str(wei_acc)+" # "+ "Scores: "+str(wei_scores))
    print("Bld: Accuracy: " + str(blood_acc)+" # "+ "Scores: "+str(blood_scores))
    print("Ask: Accuracy: " + str(ask_acc)+" # "+ "Scores: "+str(ask_scores))
    res_Stats=res_Stats.append({'Estimators':estimators,'Accuracy':res_acc,'Precision':res_scores[0],'Recall':res_scores[1],'FScore':res_scores[2]},ignore_index=True)
    wei_Stats=wei_Stats.append({'Estimators':estimators,'Accuracy':wei_acc,'Precision':wei_scores[0],'Recall':wei_scores[1],'FScore':wei_scores[2]},ignore_index=True)
    bld_Stats=bld_Stats.append({'Estimators':estimators,'Accuracy':blood_acc,'Precision':blood_scores[0],'Recall':blood_scores[1],'FScore':blood_scores[2]},ignore_index=True)
    ask_Stats=ask_Stats.append({'Estimators':estimators,'Accuracy':ask_acc,'Precision':ask_scores[0],'Recall':ask_scores[1],'FScore':ask_scores[2]},ignore_index=True)
writer = pd.ExcelWriter('Forrest_Stats'+label+'.xlsx', engine='xlsxwriter')
res_Stats.to_excel(writer, sheet_name='Sheet1') 
wei_Stats.to_excel(writer, sheet_name='Sheet2') 
bld_Stats.to_excel(writer, sheet_name='Sheet3') 
ask_Stats.to_excel(writer, sheet_name='Sheet4')
writer.save()

100
Counter({'Atenolol': 620, 'Lisinopril': 428, 'Amlodipine': 161, 'Hydrochlorothiazide': 129, 'Nadolol': 119, 'Diltiazem': 100})
Counter({'Lisinopril': 51, 'Atenolol': 45, 'Amlodipine': 14, 'Hydrochlorothiazide': 13})
Counter({'Atenolol': 186, 'Lisinopril': 134, 'Amlodipine': 63, 'Hydrochlorothiazide': 51, 'Diltiazem': 26})
Counter({'Lisinopril': 622, 'Atenolol': 134})
Counter({0: 620, 2: 620, 3: 620, 1: 620, 4: 620, 5: 620})
Counter({0: 51, 2: 51, 1: 51, 3: 51})
Counter({0: 186, 2: 186, 3: 186, 1: 186, 4: 186})
Counter({1: 622, 0: 622})
Res: Accuracy: 0.532258064516129 # Scores: (0.5274187051505063, 0.5336865809113508, 0.5294807132566177, None)
Wei: Accuracy: 0.6129032258064516 # Scores: (0.6031746031746033, 0.614390756302521, 0.6046644088669951, None)
Bld: Accuracy: 0.6164874551971327 # Scores: (0.607253837684224, 0.6197660818713451, 0.6115274728278199, None)
Ask: Accuracy: 0.7994652406417112 # Scores: (0.8002863688430699, 0.8000028609847509, 0.799452336862877, None)
200
Counter({'

Counter({'Lisinopril': 622, 'Atenolol': 134})
Counter({0: 620, 2: 620, 3: 620, 1: 620, 4: 620, 5: 620})
Counter({0: 51, 2: 51, 1: 51, 3: 51})
Counter({0: 186, 2: 186, 3: 186, 1: 186, 4: 186})
Counter({1: 622, 0: 622})
Res: Accuracy: 0.5259856630824373 # Scores: (0.5211607584692192, 0.5272502426646841, 0.5230874401122513, None)
Wei: Accuracy: 0.6290322580645161 # Scores: (0.6141825151113077, 0.6290966386554622, 0.6179308579197345, None)
Bld: Accuracy: 0.6129032258064516 # Scores: (0.6050995458600276, 0.6166276803118909, 0.6082058447071086, None)
Ask: Accuracy: 0.8021390374331551 # Scores: (0.8031514109726401, 0.8027351014219095, 0.8021164021164022, None)


In [198]:
label='Drug'
asklabel='Drug'
res_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
wei_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
bld_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
ask_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
for estimators in range (100,1000,100):
    print (estimators)
    result,result_labels,weighted,weighted_labels,blood,blood_labels,ask,ask_labels=prepareDataset(label,asklabel)
    res_feat=importance(result,result_labels)
    wei_feat=importance(weighted,weighted_labels)
    blood_feat=importance(blood,blood_labels)
    ask_feat=importance(ask,ask_labels)
    res_acc,res_scores=randomForest(pd.DataFrame(result)[res_feat],result_labels,estimators,"Result")
    wei_acc,wei_scores=randomForest(pd.DataFrame(weighted)[wei_feat],weighted_labels,estimators,"Weighted")
    blood_acc,blood_scores=randomForest(pd.DataFrame(blood)[blood_feat],blood_labels,estimators,"Blood")
    ask_acc,ask_scores=randomForest(pd.DataFrame(ask)[ask_feat],ask_labels,estimators,"Ask")
    print("Res: Accuracy: " + str(res_acc)+" # "+ "Scores: "+str(res_scores))
    print("Wei: Accuracy: " + str(wei_acc)+" # "+ "Scores: "+str(wei_scores))
    print("Bld: Accuracy: " + str(blood_acc)+" # "+ "Scores: "+str(blood_scores))
    print("Ask: Accuracy: " + str(ask_acc)+" # "+ "Scores: "+str(ask_scores))
    res_Stats=res_Stats.append({'Estimators':estimators,'Accuracy':res_acc,'Precision':res_scores[0],'Recall':res_scores[1],'FScore':res_scores[2]},ignore_index=True)
    wei_Stats=wei_Stats.append({'Estimators':estimators,'Accuracy':wei_acc,'Precision':wei_scores[0],'Recall':wei_scores[1],'FScore':wei_scores[2]},ignore_index=True)
    bld_Stats=bld_Stats.append({'Estimators':estimators,'Accuracy':blood_acc,'Precision':blood_scores[0],'Recall':blood_scores[1],'FScore':blood_scores[2]},ignore_index=True)
    ask_Stats=ask_Stats.append({'Estimators':estimators,'Accuracy':ask_acc,'Precision':ask_scores[0],'Recall':ask_scores[1],'FScore':ask_scores[2]},ignore_index=True)
writer = pd.ExcelWriter('Forrest_Stats'+label+'importance.xlsx', engine='xlsxwriter')
res_Stats.to_excel(writer, sheet_name='Sheet1') 
wei_Stats.to_excel(writer, sheet_name='Sheet2') 
bld_Stats.to_excel(writer, sheet_name='Sheet3') 
ask_Stats.to_excel(writer, sheet_name='Sheet4')
writer.save()

100
Counter({'Atenolol': 620, 'Lisinopril': 428, 'Amlodipine': 161, 'Hydrochlorothiazide': 129, 'Nadolol': 119, 'Diltiazem': 100})
Counter({'Lisinopril': 51, 'Atenolol': 45, 'Amlodipine': 14, 'Hydrochlorothiazide': 13})
Counter({'Atenolol': 186, 'Lisinopril': 134, 'Amlodipine': 63, 'Hydrochlorothiazide': 51, 'Diltiazem': 26})
Counter({'Lisinopril': 622, 'Atenolol': 134})


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when 

Counter({0: 620, 2: 620, 3: 620, 1: 620, 4: 620, 5: 620})
Counter({0: 51, 2: 51, 1: 51, 3: 51})
Counter({0: 186, 2: 186, 3: 186, 1: 186, 4: 186})
Counter({1: 622, 0: 622})
Res: Accuracy: 0.3387096774193548 # Scores: (0.33879970181783586, 0.3393305516790535, 0.33744015641691805, None)
Wei: Accuracy: 0.6290322580645161 # Scores: (0.6137723387723388, 0.6346288515406162, 0.6148303765950824, None)
Bld: Accuracy: 0.5878136200716846 # Scores: (0.5772442244018695, 0.5896003898635478, 0.5796191598936657, None)
Ask: Accuracy: 0.8288770053475936 # Scores: (0.8393965491640187, 0.8269676422624668, 0.8268969307761289, None)
200
Counter({'Atenolol': 620, 'Lisinopril': 428, 'Amlodipine': 161, 'Hydrochlorothiazide': 129, 'Nadolol': 119, 'Diltiazem': 100})
Counter({'Lisinopril': 51, 'Atenolol': 45, 'Amlodipine': 14, 'Hydrochlorothiazide': 13})
Counter({'Atenolol': 186, 'Lisinopril': 134, 'Amlodipine': 63, 'Hydrochlorothiazide': 51, 'Diltiazem': 26})
Counter({'Lisinopril': 622, 'Atenolol': 134})


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when 

Counter({0: 620, 2: 620, 3: 620, 1: 620, 4: 620, 5: 620})
Counter({0: 51, 2: 51, 1: 51, 3: 51})
Counter({0: 186, 2: 186, 3: 186, 1: 186, 4: 186})
Counter({1: 622, 0: 622})
Res: Accuracy: 0.31451612903225806 # Scores: (0.3175477425149616, 0.3155503464699486, 0.3132515298274383, None)
Wei: Accuracy: 0.6451612903225806 # Scores: (0.6079398864809081, 0.6523371848739495, 0.6180555555555556, None)
Bld: Accuracy: 0.5913978494623656 # Scores: (0.5821821036106749, 0.5939424951267056, 0.5817597866537285, None)
Ask: Accuracy: 0.8262032085561497 # Scores: (0.8359708392603129, 0.8243498412153463, 0.8242925806801344, None)
300
Counter({'Atenolol': 620, 'Lisinopril': 428, 'Amlodipine': 161, 'Hydrochlorothiazide': 129, 'Nadolol': 119, 'Diltiazem': 100})
Counter({'Lisinopril': 51, 'Atenolol': 45, 'Amlodipine': 14, 'Hydrochlorothiazide': 13})
Counter({'Atenolol': 186, 'Lisinopril': 134, 'Amlodipine': 63, 'Hydrochlorothiazide': 51, 'Diltiazem': 26})
Counter({'Lisinopril': 622, 'Atenolol': 134})


...

Counter({0: 620, 2: 620, 3: 620, 1: 620, 4: 620, 5: 620})
Counter({0: 51, 2: 51, 1: 51, 3: 51})
Counter({0: 186, 2: 186, 3: 186, 1: 186, 4: 186})
Counter({1: 622, 0: 622})
Res: Accuracy: 0.3342293906810036 # Scores: (0.3315681336274718, 0.33489680184155873, 0.33142196148586933, None)
Wei: Accuracy: 0.6612903225806451 # Scores: (0.6348039215686274, 0.6679621848739495, 0.6407142857142857, None)
Bld: Accuracy: 0.5913978494623656 # Scores: (0.5829153332543162, 0.5931091617933724, 0.5823535513592776, None)
Ask: Accuracy: 0.8262032085561497 # Scores: (0.8359708392603129, 0.8243498412153463, 0.8242925806801344, None)
800
Counter({'Atenolol': 620, 'Lisinopril': 428, 'Amlodipine': 161, 'Hydrochlorothiazide': 129, 'Nadolol': 119, 'Diltiazem': 100})
Counter({'Lisinopril': 51, 'Atenolol': 45, 'Amlodipine': 14, 'Hydrochlorothiazide': 13})
Counter({'Atenolol': 186, 'Lisinopril': 134, 'Amlodipine': 63, 'Hydrochlorothiazide': 51, 'Diltiazem': 26})
Counter({'Lisinopril': 622, 'Atenolol': 134})


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when 

Counter({0: 620, 2: 620, 3: 620, 1: 620, 4: 620, 5: 620})
Counter({0: 51, 2: 51, 1: 51, 3: 51})
Counter({0: 186, 2: 186, 3: 186, 1: 186, 4: 186})
Counter({1: 622, 0: 622})
Res: Accuracy: 0.3342293906810036 # Scores: (0.3320379476828122, 0.33490643714531165, 0.3315934403882476, None)
Wei: Accuracy: 0.6612903225806451 # Scores: (0.6480493191019507, 0.6681109943977591, 0.6486892736892736, None)
Bld: Accuracy: 0.5842293906810035 # Scores: (0.5742032120421952, 0.5860721247563353, 0.5745858288856728, None)
Ask: Accuracy: 0.8288770053475936 # Scores: (0.8369126528207395, 0.8271965210425429, 0.8272766632991775, None)
900
Counter({'Atenolol': 620, 'Lisinopril': 428, 'Amlodipine': 161, 'Hydrochlorothiazide': 129, 'Nadolol': 119, 'Diltiazem': 100})
Counter({'Lisinopril': 51, 'Atenolol': 45, 'Amlodipine': 14, 'Hydrochlorothiazide': 13})
Counter({'Atenolol': 186, 'Lisinopril': 134, 'Amlodipine': 63, 'Hydrochlorothiazide': 51, 'Diltiazem': 26})
Counter({'Lisinopril': 622, 'Atenolol': 134})


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when 

Counter({0: 620, 2: 620, 3: 620, 1: 620, 4: 620, 5: 620})
Counter({0: 51, 2: 51, 1: 51, 3: 51})
Counter({0: 186, 2: 186, 3: 186, 1: 186, 4: 186})
Counter({1: 622, 0: 622})
Res: Accuracy: 0.3172043010752688 # Scores: (0.32111226872933624, 0.318137479114361, 0.3161647694351423, None)
Wei: Accuracy: 0.6612903225806451 # Scores: (0.6348039215686274, 0.6679621848739495, 0.6407142857142857, None)
Bld: Accuracy: 0.5878136200716846 # Scores: (0.5766196000020614, 0.5895808966861599, 0.5770596315214812, None)
Ask: Accuracy: 0.8262032085561497 # Scores: (0.8359708392603129, 0.8243498412153463, 0.8242925806801344, None)


In [199]:
label='Drug'
asklabel='Drug'
res_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
wei_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
bld_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
ask_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
for estimators in range (100,1000,100):
    print (estimators)
    result,result_labels,weighted,weighted_labels,blood,blood_labels,ask,ask_labels=prepareDatasetNoSmote(label,asklabel)
    res_acc,res_scores=randomForest(result,result_labels,estimators,"Result")
    wei_acc,wei_scores=randomForest(weighted,weighted_labels,estimators,"Weighted")
    blood_acc,blood_scores=randomForest(blood,blood_labels,estimators,"Blood")
    ask_acc,ask_scores=randomForest(ask,ask_labels,estimators,"Ask")
    print("Res: Accuracy: " + str(res_acc)+" # "+ "Scores: "+str(res_scores))
    print("Wei: Accuracy: " + str(wei_acc)+" # "+ "Scores: "+str(wei_scores))
    print("Bld: Accuracy: " + str(blood_acc)+" # "+ "Scores: "+str(blood_scores))
    print("Ask: Accuracy: " + str(ask_acc)+" # "+ "Scores: "+str(ask_scores))
    res_Stats=res_Stats.append({'Estimators':estimators,'Accuracy':res_acc,'Precision':res_scores[0],'Recall':res_scores[1],'FScore':res_scores[2]},ignore_index=True)
    wei_Stats=wei_Stats.append({'Estimators':estimators,'Accuracy':wei_acc,'Precision':wei_scores[0],'Recall':wei_scores[1],'FScore':wei_scores[2]},ignore_index=True)
    bld_Stats=bld_Stats.append({'Estimators':estimators,'Accuracy':blood_acc,'Precision':blood_scores[0],'Recall':blood_scores[1],'FScore':blood_scores[2]},ignore_index=True)
    ask_Stats=ask_Stats.append({'Estimators':estimators,'Accuracy':ask_acc,'Precision':ask_scores[0],'Recall':ask_scores[1],'FScore':ask_scores[2]},ignore_index=True)
writer = pd.ExcelWriter('Forrest_Stats'+label+'smoteless.xlsx', engine='xlsxwriter')
res_Stats.to_excel(writer, sheet_name='Sheet1') 
wei_Stats.to_excel(writer, sheet_name='Sheet2') 
bld_Stats.to_excel(writer, sheet_name='Sheet3') 
ask_Stats.to_excel(writer, sheet_name='Sheet4')
writer.save()

100
Counter({'Atenolol': 620, 'Lisinopril': 428, 'Amlodipine': 161, 'Hydrochlorothiazide': 129, 'Nadolol': 119, 'Diltiazem': 100})
Counter({'Lisinopril': 51, 'Atenolol': 45, 'Amlodipine': 14, 'Hydrochlorothiazide': 13})
Counter({'Atenolol': 186, 'Lisinopril': 134, 'Amlodipine': 63, 'Hydrochlorothiazide': 51, 'Diltiazem': 26})
Counter({'Lisinopril': 622, 'Atenolol': 134})
Counter({1: 620, 4: 428, 0: 161, 3: 129, 5: 119, 2: 100})
Counter({3: 51, 1: 45, 0: 14, 2: 13})
Counter({1: 186, 4: 134, 0: 63, 3: 51, 2: 26})
Counter({1: 622, 0: 134})
Res: Accuracy: 0.3055555555555556 # Scores: (0.1830162210618924, 0.1877610513191222, 0.18294342166741184, None)
Wei: Accuracy: 0.4864864864864865 # Scores: (0.265625, 0.3375, 0.29707792207792205, None)
Bld: Accuracy: 0.3695652173913043 # Scores: (0.2984892300681774, 0.2624586363716798, 0.25506949191159717, None)
Ask: Accuracy: 0.7665198237885462 # Scores: (0.5437975347495411, 0.5599425699928212, 0.5471108601543384, None)
200
Counter({'Atenolol': 620, 'L

Counter({'Lisinopril': 622, 'Atenolol': 134})
Counter({1: 620, 4: 428, 0: 161, 3: 129, 5: 119, 2: 100})
Counter({3: 51, 1: 45, 0: 14, 2: 13})
Counter({1: 186, 4: 134, 0: 63, 3: 51, 2: 26})
Counter({1: 622, 0: 134})
Res: Accuracy: 0.3076923076923077 # Scores: (0.18374439880720705, 0.1863117204734659, 0.18181003525254683, None)
Wei: Accuracy: 0.4594594594594595 # Scores: (0.2583333333333333, 0.31666666666666665, 0.2845117845117845, None)
Bld: Accuracy: 0.36231884057971014 # Scores: (0.2897566810596336, 0.2581108102847233, 0.2502266844458389, None)
Ask: Accuracy: 0.7841409691629956 # Scores: (0.5554765291607398, 0.5699928212491027, 0.5600949179355349, None)


In [193]:
label='Drug'
asklabel='Drug'
res_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
wei_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
bld_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
ask_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
for estimators in range (100,1000,100):
    print (estimators)
    result,result_labels,weighted,weighted_labels,blood,blood_labels,ask,ask_labels=prepareDatasetNoSmote(label,asklabel)
    res_feat=importance(result,result_labels)
    wei_feat=importance(weighted,weighted_labels)
    blood_feat=importance(blood,blood_labels)
    ask_feat=importance(ask,ask_labels)
    res_acc,res_scores=randomForest(pd.DataFrame(result)[res_feat],result_labels,estimators,"Result")
    wei_acc,wei_scores=randomForest(pd.DataFrame(weighted)[wei_feat],weighted_labels,estimators,"Weighted")
    blood_acc,blood_scores=randomForest(pd.DataFrame(blood)[blood_feat],blood_labels,estimators,"Blood")
    ask_acc,ask_scores=randomForest(pd.DataFrame(ask)[ask_feat],ask_labels,estimators,"Ask")
    print("Res: Accuracy: " + str(res_acc)+" # "+ "Scores: "+str(res_scores))
    print("Wei: Accuracy: " + str(wei_acc)+" # "+ "Scores: "+str(wei_scores))
    print("Bld: Accuracy: " + str(blood_acc)+" # "+ "Scores: "+str(blood_scores))
    print("Ask: Accuracy: " + str(ask_acc)+" # "+ "Scores: "+str(ask_scores))
    res_Stats=res_Stats.append({'Estimators':estimators,'Accuracy':res_acc,'Precision':res_scores[0],'Recall':res_scores[1],'FScore':res_scores[2]},ignore_index=True)
    wei_Stats=wei_Stats.append({'Estimators':estimators,'Accuracy':wei_acc,'Precision':wei_scores[0],'Recall':wei_scores[1],'FScore':wei_scores[2]},ignore_index=True)
    bld_Stats=bld_Stats.append({'Estimators':estimators,'Accuracy':blood_acc,'Precision':blood_scores[0],'Recall':blood_scores[1],'FScore':blood_scores[2]},ignore_index=True)
    ask_Stats=ask_Stats.append({'Estimators':estimators,'Accuracy':ask_acc,'Precision':ask_scores[0],'Recall':ask_scores[1],'FScore':ask_scores[2]},ignore_index=True)

writer = pd.ExcelWriter('Forrest_Stats'+label+'importancesmoteless.xlsx', engine='xlsxwriter')
res_Stats.to_excel(writer, sheet_name='Sheet1') 
wei_Stats.to_excel(writer, sheet_name='Sheet2') 
bld_Stats.to_excel(writer, sheet_name='Sheet3') 
ask_Stats.to_excel(writer, sheet_name='Sheet4')
writer.save()

100
Counter({'Atenolol': 620, 'Lisinopril': 428, 'Amlodipine': 161, 'Hydrochlorothiazide': 129, 'Nadolol': 119, 'Diltiazem': 100})
Counter({'Lisinopril': 51, 'Atenolol': 45, 'Amlodipine': 14, 'Hydrochlorothiazide': 13})
Counter({'Atenolol': 186, 'Lisinopril': 134, 'Amlodipine': 63, 'Hydrochlorothiazide': 51, 'Diltiazem': 26})
Counter({'Lisinopril': 622, 'Atenolol': 134})


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when 

Counter({1: 620, 4: 428, 0: 161, 3: 129, 5: 119, 2: 100})
Counter({3: 51, 1: 45, 0: 14, 2: 13})
Counter({1: 186, 4: 134, 0: 63, 3: 51, 2: 26})
Counter({1: 622, 0: 134})
Res: Accuracy: 0.33547008547008544 # Scores: (0.21619569828839488, 0.2066556538341315, 0.201388500124543, None)
Wei: Accuracy: 0.43243243243243246 # Scores: (0.2957983193277311, 0.3291666666666667, 0.3086206896551724, None)
Bld: Accuracy: 0.34057971014492755 # Scores: (0.23772910188403146, 0.24122737166215424, 0.2304187120931307, None)
Ask: Accuracy: 0.8061674008810573 # Scores: (0.5518664752333093, 0.5518664752333093, 0.5518664752333093, None)
200
Counter({'Atenolol': 620, 'Lisinopril': 428, 'Amlodipine': 161, 'Hydrochlorothiazide': 129, 'Nadolol': 119, 'Diltiazem': 100})
Counter({'Lisinopril': 51, 'Atenolol': 45, 'Amlodipine': 14, 'Hydrochlorothiazide': 13})
Counter({'Atenolol': 186, 'Lisinopril': 134, 'Amlodipine': 63, 'Hydrochlorothiazide': 51, 'Diltiazem': 26})
Counter({'Lisinopril': 622, 'Atenolol': 134})


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when 

Counter({1: 620, 4: 428, 0: 161, 3: 129, 5: 119, 2: 100})
Counter({3: 51, 1: 45, 0: 14, 2: 13})
Counter({1: 186, 4: 134, 0: 63, 3: 51, 2: 26})
Counter({1: 622, 0: 134})
Res: Accuracy: 0.344017094017094 # Scores: (0.22472291806442082, 0.21524432384012437, 0.20866198475619255, None)
Wei: Accuracy: 0.43243243243243246 # Scores: (0.3023785425101214, 0.33333333333333337, 0.3095878136200717, None)
Bld: Accuracy: 0.3333333333333333 # Scores: (0.24312573443008226, 0.23765992461644636, 0.22989323786667093, None)
Ask: Accuracy: 0.7929515418502202 # Scores: (0.5406517445687953, 0.5443287867910983, 0.5422012443681613, None)
300
Counter({'Atenolol': 620, 'Lisinopril': 428, 'Amlodipine': 161, 'Hydrochlorothiazide': 129, 'Nadolol': 119, 'Diltiazem': 100})
Counter({'Lisinopril': 51, 'Atenolol': 45, 'Amlodipine': 14, 'Hydrochlorothiazide': 13})
Counter({'Atenolol': 186, 'Lisinopril': 134, 'Amlodipine': 63, 'Hydrochlorothiazide': 51, 'Diltiazem': 26})
Counter({'Lisinopril': 622, 'Atenolol': 134})


...

Counter({1: 620, 4: 428, 0: 161, 3: 129, 5: 119, 2: 100})
Counter({3: 51, 1: 45, 0: 14, 2: 13})
Counter({1: 186, 4: 134, 0: 63, 3: 51, 2: 26})
Counter({1: 622, 0: 134})
Res: Accuracy: 0.3418803418803419 # Scores: (0.23171533080156723, 0.2147929913677945, 0.20760151246838285, None)
Wei: Accuracy: 0.43243243243243246 # Scores: (0.24812030075187969, 0.3, 0.26696329254727474, None)
Bld: Accuracy: 0.3188405797101449 # Scores: (0.21801619433198377, 0.22740351436003609, 0.21604516341358443, None)
Ask: Accuracy: 0.7797356828193832 # Scores: (0.5312404754647974, 0.5367910983488873, 0.5329986833443054, None)
800
Counter({'Atenolol': 620, 'Lisinopril': 428, 'Amlodipine': 161, 'Hydrochlorothiazide': 129, 'Nadolol': 119, 'Diltiazem': 100})
Counter({'Lisinopril': 51, 'Atenolol': 45, 'Amlodipine': 14, 'Hydrochlorothiazide': 13})
Counter({'Atenolol': 186, 'Lisinopril': 134, 'Amlodipine': 63, 'Hydrochlorothiazide': 51, 'Diltiazem': 26})
Counter({'Lisinopril': 622, 'Atenolol': 134})


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when 

Counter({1: 620, 4: 428, 0: 161, 3: 129, 5: 119, 2: 100})
Counter({3: 51, 1: 45, 0: 14, 2: 13})
Counter({1: 186, 4: 134, 0: 63, 3: 51, 2: 26})
Counter({1: 622, 0: 134})
Res: Accuracy: 0.3482905982905983 # Scores: (0.21603972472874913, 0.20997544361225462, 0.2019383388365942, None)
Wei: Accuracy: 0.43243243243243246 # Scores: (0.3177083333333333, 0.36250000000000004, 0.3365987460815047, None)
Bld: Accuracy: 0.3188405797101449 # Scores: (0.21586770717205503, 0.22740351436003609, 0.21539452495974235, None)
Ask: Accuracy: 0.7973568281938326 # Scores: (0.5441624365482234, 0.5468413496051687, 0.5453674677812609, None)
900
Counter({'Atenolol': 620, 'Lisinopril': 428, 'Amlodipine': 161, 'Hydrochlorothiazide': 129, 'Nadolol': 119, 'Diltiazem': 100})
Counter({'Lisinopril': 51, 'Atenolol': 45, 'Amlodipine': 14, 'Hydrochlorothiazide': 13})
Counter({'Atenolol': 186, 'Lisinopril': 134, 'Amlodipine': 63, 'Hydrochlorothiazide': 51, 'Diltiazem': 26})
Counter({'Lisinopril': 622, 'Atenolol': 134})


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when 

Counter({1: 620, 4: 428, 0: 161, 3: 129, 5: 119, 2: 100})
Counter({3: 51, 1: 45, 0: 14, 2: 13})
Counter({1: 186, 4: 134, 0: 63, 3: 51, 2: 26})
Counter({1: 622, 0: 134})
Res: Accuracy: 0.3504273504273504 # Scores: (0.20486454971047538, 0.21085727782742222, 0.2022710861274368, None)
Wei: Accuracy: 0.43243243243243246 # Scores: (0.24812030075187969, 0.3, 0.26696329254727474, None)
Bld: Accuracy: 0.3188405797101449 # Scores: (0.21436620567055353, 0.22740351436003609, 0.21469277057377742, None)
Ask: Accuracy: 0.7973568281938326 # Scores: (0.5441624365482234, 0.5468413496051687, 0.5453674677812609, None)


### Random Forest Classifier Function 10-Fold

In [73]:
def average(scores):
    FScore=0
    Recall=0
    Precision=0

    for i in scores:
        Precision+=i[0]
        Recall+=i[1]
        FScore+=i[2]
    return (Precision/10,Recall/10,FScore/10)

In [134]:
def randomForest10Fold(result,result_labels,estimators,DataSet):
    from sklearn.model_selection import KFold
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import classification_report
    from sklearn.metrics import f1_score
    from collections import Counter
    kf = KFold(n_splits=10)

    accuracy=[]

    scores_rf=[]
    print(Counter(result_labels))
    count = 1
    for train_index, test_index in kf.split(result):
        X_train, X_test, Y_train, Y_test = result[train_index], result[test_index], result_labels[train_index],result_labels[test_index]
        clf = RandomForestClassifier(n_estimators=estimators, max_depth=2,random_state=0)
        clf.fit(X_train, Y_train)
        pred=clf.predict(X_test)
        accuracy.append(accuracy_score(Y_test, pred))
        scores_rf.append(precision_recall_fscore_support(Y_test,pred, average='macro'))
        
        #print(str(count)+DataSet)
        #print(f1_score(Y_test, pred, average='micro'))
        count+=1
        #print (pred)
        #print(classification_report(Y_test,pred,labels=[0,1]))
    accuracy = np.mean(accuracy,axis=0)   
    return accuracy,average(scores_rf)

In [135]:
def randomForest10FoldNotSmote(result,result_labels,estimators,DataSet):
    from sklearn.model_selection import KFold
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import classification_report
    from sklearn.metrics import f1_score
    from collections import Counter
    kf = KFold(n_splits=10)

    accuracy=[]

    scores_rf=[]
    print(Counter(result_labels))
    count = 1
    for train_index, test_index in kf.split(result):
        X_train, X_test, Y_train, Y_test = result.iloc[train_index], result.iloc[test_index], result_labels[train_index],result_labels[test_index]
        clf = RandomForestClassifier(n_estimators=estimators, max_depth=2,random_state=0)
        clf.fit(X_train, Y_train)
        pred=clf.predict(X_test)
        accuracy.append(accuracy_score(Y_test, pred))
        scores_rf.append(precision_recall_fscore_support(Y_test,pred, average='macro'))
        
        #print(str(count)+DataSet)
        #print(f1_score(Y_test, pred, average='micro'))
        count+=1
        #print (pred)
        #print(classification_report(Y_test,pred,labels=[0,1]))
    accuracy = np.mean(accuracy,axis=0)   
    return accuracy,average(scores_rf)

### Random Forest with 10-Fold Cross Validation

In [149]:
label='Drug'
asklabel='Drug'

res_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
wei_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
bld_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
ask_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
for estimators in range (100,1000,100):
    result,result_labels,weighted,weighted_labels,blood,blood_labels,ask,ask_labels=prepareDataset(label,asklabel)
    res_acc,res_scores=randomForest10Fold(result,result_labels,estimators,"Result")
    wei_acc,wei_scores=randomForest10Fold(weighted,weighted_labels,estimators,"Weighted")
    blood_acc,blood_scores=randomForest10Fold(blood,blood_labels,estimators,"Blood")
    ask_acc,ask_scores=randomForest10Fold(ask,ask_labels,1000,"Ask")
    print("Res: Accuracy: " + str(res_acc)+" # "+ "Scores: "+str(res_scores))
    print("Wei: Accuracy: " + str(wei_acc)+" # "+ "Scores: "+str(wei_scores))
    print("Bld: Accuracy: " + str(blood_acc)+" # "+ "Scores: "+str(blood_scores))
    print("Ask: Accuracy: " + str(ask_acc)+" # "+ "Scores: "+str(ask_scores))
    res_Stats=res_Stats.append({'Estimators':estimators,'Accuracy':res_acc,'Precision':res_scores[0],'Recall':res_scores[1],'FScore':res_scores[2]},ignore_index=True)
    wei_Stats=wei_Stats.append({'Estimators':estimators,'Accuracy':wei_acc,'Precision':wei_scores[0],'Recall':wei_scores[1],'FScore':wei_scores[2]},ignore_index=True)
    bld_Stats=bld_Stats.append({'Estimators':estimators,'Accuracy':blood_acc,'Precision':blood_scores[0],'Recall':blood_scores[1],'FScore':blood_scores[2]},ignore_index=True)
    ask_Stats=ask_Stats.append({'Estimators':estimators,'Accuracy':ask_acc,'Precision':ask_scores[0],'Recall':ask_scores[1],'FScore':ask_scores[2]},ignore_index=True)
writer = pd.ExcelWriter('ForrestFold_Stats'+label+'.xlsx', engine='xlsxwriter')
res_Stats.to_excel(writer, sheet_name='Sheet1') 
wei_Stats.to_excel(writer, sheet_name='Sheet2') 
bld_Stats.to_excel(writer, sheet_name='Sheet3') 
ask_Stats.to_excel(writer, sheet_name='Sheet4')
writer.save()

Counter({'Atenolol': 620, 'Lisinopril': 428, 'Amlodipine': 161, 'Hydrochlorothiazide': 129, 'Nadolol': 119, 'Diltiazem': 100})
Counter({'Lisinopril': 51, 'Atenolol': 45, 'Amlodipine': 14, 'Hydrochlorothiazide': 13})
Counter({'Atenolol': 186, 'Lisinopril': 134, 'Amlodipine': 63, 'Hydrochlorothiazide': 51, 'Diltiazem': 26})
Counter({'Lisinopril': 622, 'Atenolol': 134})
Counter({0: 620, 2: 620, 3: 620, 1: 620, 4: 620, 5: 620})


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\sit

Counter({0: 51, 2: 51, 1: 51, 3: 51})


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\class

Counter({0: 186, 2: 186, 3: 186, 1: 186, 4: 186})


...

C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\sit

Counter({1: 622, 0: 622})


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\class

Res: Accuracy: 0.030376344086021505 # Scores: (0.08485316093525049, 0.021884115116468657, 0.024143211786656606)
Wei: Accuracy: 0.19666666666666666 # Scores: (0.2751190476190476, 0.12057174185463657, 0.12042401803177664)
Bld: Accuracy: 0.0870967741935484 # Scores: (0.12715138195138195, 0.08681380484068255, 0.06265369508775241)
Ask: Accuracy: 0.6475612903225806 # Scores: (0.5025641025641026, 0.37353606557377045, 0.3928535487901206)


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [190]:
label='Drug'
asklabel='Drug'

res_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
wei_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
bld_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
ask_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
for estimators in range (100,1000,100):
    result,result_labels,weighted,weighted_labels,blood,blood_labels,ask,ask_labels=prepareDatasetNoSmote(label,asklabel)
    res_acc,res_scores=randomForest10FoldNotSmote(result,result_labels,estimators,"Result")
    wei_acc,wei_scores=randomForest10FoldNotSmote(weighted,weighted_labels,estimators,"Weighted")
    blood_acc,blood_scores=randomForest10FoldNotSmote(blood,blood_labels,estimators,"Blood")
    ask_acc,ask_scores=randomForest10FoldNotSmote(ask,ask_labels,1000,"Ask")
    print("Res: Accuracy: " + str(res_acc)+" # "+ "Scores: "+str(res_scores))
    print("Wei: Accuracy: " + str(wei_acc)+" # "+ "Scores: "+str(wei_scores))
    print("Bld: Accuracy: " + str(blood_acc)+" # "+ "Scores: "+str(blood_scores))
    print("Ask: Accuracy: " + str(ask_acc)+" # "+ "Scores: "+str(ask_scores))
    res_Stats=res_Stats.append({'Estimators':estimators,'Accuracy':res_acc,'Precision':res_scores[0],'Recall':res_scores[1],'FScore':res_scores[2]},ignore_index=True)
    wei_Stats=wei_Stats.append({'Estimators':estimators,'Accuracy':wei_acc,'Precision':wei_scores[0],'Recall':wei_scores[1],'FScore':wei_scores[2]},ignore_index=True)
    bld_Stats=bld_Stats.append({'Estimators':estimators,'Accuracy':blood_acc,'Precision':blood_scores[0],'Recall':blood_scores[1],'FScore':blood_scores[2]},ignore_index=True)
    ask_Stats=ask_Stats.append({'Estimators':estimators,'Accuracy':ask_acc,'Precision':ask_scores[0],'Recall':ask_scores[1],'FScore':ask_scores[2]},ignore_index=True)

writer = pd.ExcelWriter('ForrestFold_Stats'+label+'smoteless.xlsx', engine='xlsxwriter')
res_Stats.to_excel(writer, sheet_name='Sheet1') 
wei_Stats.to_excel(writer, sheet_name='Sheet2') 
bld_Stats.to_excel(writer, sheet_name='Sheet3') 
ask_Stats.to_excel(writer, sheet_name='Sheet4')
writer.save()

Counter({'Atenolol': 620, 'Lisinopril': 428, 'Amlodipine': 161, 'Hydrochlorothiazide': 129, 'Nadolol': 119, 'Diltiazem': 100})
Counter({'Lisinopril': 51, 'Atenolol': 45, 'Amlodipine': 14, 'Hydrochlorothiazide': 13})
Counter({'Atenolol': 186, 'Lisinopril': 134, 'Amlodipine': 63, 'Hydrochlorothiazide': 51, 'Diltiazem': 26})
Counter({'Lisinopril': 622, 'Atenolol': 134})
Counter({1: 620, 4: 428, 0: 161, 3: 129, 5: 119, 2: 100})


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\sit

Counter({3: 51, 1: 45, 0: 14, 2: 13})


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\sit

Counter({1: 186, 4: 134, 0: 63, 3: 51, 2: 26})


...

C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\sit

Counter({1: 186, 4: 134, 0: 63, 3: 51, 2: 26})


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\sit

Counter({1: 622, 0: 134})


KeyboardInterrupt: 

In [183]:
result,result_labels,weighted,weighted_labels,blood,blood_labels,ask,ask_labels=prepareDataset(label,asklabel)

Counter({'Atenolol': 620, 'Lisinopril': 428, 'Amlodipine': 161, 'Hydrochlorothiazide': 129, 'Nadolol': 119, 'Diltiazem': 100})
Counter({'Lisinopril': 51, 'Atenolol': 45, 'Amlodipine': 14, 'Hydrochlorothiazide': 13})
Counter({'Atenolol': 186, 'Lisinopril': 134, 'Amlodipine': 63, 'Hydrochlorothiazide': 51, 'Diltiazem': 26})
Counter({'Lisinopril': 622, 'Atenolol': 134})


### SVM Classifier Function

In [54]:
def SVM(result,result_labels,DataSet):
    from sklearn.model_selection import KFold
    from sklearn.svm import SVC
    from sklearn.metrics import accuracy_score
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report
    from sklearn.metrics import f1_score
    from collections import Counter
    
    X_train, X_test, Y_train, Y_test = train_test_split(result, result_labels, test_size=0.30, random_state=42)
    print(Counter(result_labels))
    clf = SVC(gamma='auto')
    clf.fit(X_train, Y_train)
    pred=clf.predict(X_test)
    accuracy=(accuracy_score(Y_test, pred))
    #print (accuracy)
    scores_rf=(precision_recall_fscore_support(Y_test,pred, average='macro'))
    #print(DataSet)
    #print (pred)
    #print(classification_report(Y_test,pred,labels=[0,1,2]))
        
    return accuracy,scores_rf

### SVM  

In [151]:
label='Drug'
asklabel='Drug'
Stats = pd.DataFrame(columns = ['Dataset','Accuracy', 'Precision' , 'Recall','FScore'])


result,result_labels,weighted,weighted_labels,blood,blood_labels,ask,ask_labels=prepareDataset(label,asklabel)
res_acc,res_scores=SVM(result,result_labels,"Result")
wei_acc,wei_scores=SVM(weighted,weighted_labels,"Weighted")
blood_acc,blood_scores=SVM(blood,blood_labels,"Blood")
ask_acc,ask_scores=SVM(ask,ask_labels,"Ask")
print("Res: Accuracy: " + str(res_acc)+" # "+ "Scores: "+str(res_scores))
print("Wei: Accuracy: " + str(wei_acc)+" # "+ "Scores: "+str(wei_scores))
print("Bld: Accuracy: " + str(blood_acc)+" # "+ "Scores: "+str(blood_scores))
print("Ask: Accuracy: " + str(ask_acc)+" # "+ "Scores: "+str(ask_scores))
Stats=Stats.append({'Dataset':"Complete",'Accuracy':res_acc,'Precision':res_scores[0],'Recall':res_scores[1],'FScore':res_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"Weighted",'Accuracy':wei_acc,'Precision':wei_scores[0],'Recall':wei_scores[1],'FScore':wei_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"Blood",'Accuracy':blood_acc,'Precision':blood_scores[0],'Recall':blood_scores[1],'FScore':blood_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"AskAPatient",'Accuracy':ask_acc,'Precision':ask_scores[0],'Recall':ask_scores[1],'FScore':ask_scores[2]},ignore_index=True)
Stats.to_excel('SVMresult_Stats'+label+'.xlsx') 


Counter({'Atenolol': 620, 'Lisinopril': 428, 'Amlodipine': 161, 'Hydrochlorothiazide': 129, 'Nadolol': 119, 'Diltiazem': 100})
Counter({'Lisinopril': 51, 'Atenolol': 45, 'Amlodipine': 14, 'Hydrochlorothiazide': 13})
Counter({'Atenolol': 186, 'Lisinopril': 134, 'Amlodipine': 63, 'Hydrochlorothiazide': 51, 'Diltiazem': 26})
Counter({'Lisinopril': 622, 'Atenolol': 134})
Counter({0: 620, 2: 620, 3: 620, 1: 620, 4: 620, 5: 620})
Counter({0: 51, 2: 51, 1: 51, 3: 51})
Counter({0: 186, 2: 186, 3: 186, 1: 186, 4: 186})
Counter({1: 622, 0: 622})
Res: Accuracy: 0.43727598566308246 # Scores: (0.4408465002376966, 0.43831468465976026, 0.43491665821041336, None)
Wei: Accuracy: 0.5161290322580645 # Scores: (0.5428685897435898, 0.5167279411764706, 0.5243712695325597, None)
Bld: Accuracy: 0.5232974910394266 # Scores: (0.5215450761447806, 0.5306530214424952, 0.5201889935353436, None)
Ask: Accuracy: 0.7566844919786097 # Scores: (0.76807728781413, 0.7588046805710525, 0.7550012597631646, None)


In [176]:
label='Drug'
asklabel='Drug'
Stats = pd.DataFrame(columns = ['Dataset','Accuracy', 'Precision' , 'Recall','FScore'])


result,result_labels,weighted,weighted_labels,blood,blood_labels,ask,ask_labels=prepareDataset(label,asklabel)
res_feat=importance(result,result_labels)
wei_feat=importance(weighted,weighted_labels)
blood_feat=importance(blood,blood_labels)
ask_feat=importance(ask,ask_labels)
#res_acc,res_scores=randomForest(pd.DataFrame(result)[res_feat],result_labels,estimators,"Result")
#wei_acc,wei_scores=randomForest(pd.DataFrame(weighted)[wei_feat],weighted_labels,estimators,"Weighted")
#blood_acc,blood_scores=randomForest(pd.DataFrame(blood)[blood_feat],blood_labels,estimators,"Blood")
#ask_acc,ask_scores=randomForest(pd.DataFrame(ask)[ask_feat],ask_labels,estimators,"Ask")
res_acc,res_scores=SVM(pd.DataFrame(result)[res_feat],result_labels,"Result")
wei_acc,wei_scores=SVM(pd.DataFrame(weighted)[wei_feat],weighted_labels,"Weighted")
blood_acc,blood_scores=SVM(pd.DataFrame(blood)[blood_feat],blood_labels,"Blood")
ask_acc,ask_scores=SVM(pd.DataFrame(ask)[ask_feat],ask_labels,"Ask")
print("Res: Accuracy: " + str(res_acc)+" # "+ "Scores: "+str(res_scores))
print("Wei: Accuracy: " + str(wei_acc)+" # "+ "Scores: "+str(wei_scores))
print("Bld: Accuracy: " + str(blood_acc)+" # "+ "Scores: "+str(blood_scores))
print("Ask: Accuracy: " + str(ask_acc)+" # "+ "Scores: "+str(ask_scores))
Stats=Stats.append({'Dataset':"Complete",'Accuracy':res_acc,'Precision':res_scores[0],'Recall':res_scores[1],'FScore':res_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"Weighted",'Accuracy':wei_acc,'Precision':wei_scores[0],'Recall':wei_scores[1],'FScore':wei_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"Blood",'Accuracy':blood_acc,'Precision':blood_scores[0],'Recall':blood_scores[1],'FScore':blood_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"AskAPatient",'Accuracy':ask_acc,'Precision':ask_scores[0],'Recall':ask_scores[1],'FScore':ask_scores[2]},ignore_index=True)
Stats.to_excel('SVMresult_Stats'+label+'importance.xlsx') 

Counter({'Atenolol': 620, 'Lisinopril': 428, 'Amlodipine': 161, 'Hydrochlorothiazide': 129, 'Nadolol': 119, 'Diltiazem': 100})
Counter({'Lisinopril': 51, 'Atenolol': 45, 'Amlodipine': 14, 'Hydrochlorothiazide': 13})
Counter({'Atenolol': 186, 'Lisinopril': 134, 'Amlodipine': 63, 'Hydrochlorothiazide': 51, 'Diltiazem': 26})
Counter({'Lisinopril': 622, 'Atenolol': 134})


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when 

Counter({0: 620, 2: 620, 3: 620, 1: 620, 4: 620, 5: 620})
Counter({0: 51, 2: 51, 1: 51, 3: 51})
Counter({0: 186, 2: 186, 3: 186, 1: 186, 4: 186})
Counter({1: 622, 0: 622})
Res: Accuracy: 0.3835125448028674 # Scores: (0.3964206742974792, 0.38446806666064776, 0.38280151143771385, None)
Wei: Accuracy: 0.45161290322580644 # Scores: (0.5728174603174603, 0.4484768907563025, 0.4667992800985128, None)
Bld: Accuracy: 0.4838709677419355 # Scores: (0.5144709692434292, 0.4905701754385966, 0.48443274853801166, None)
Ask: Accuracy: 0.7513368983957219 # Scores: (0.7650980626644344, 0.7536835178668497, 0.7491399268667374, None)


In [153]:
label='Drug'
asklabel='Drug'
Stats = pd.DataFrame(columns = ['Dataset','Accuracy', 'Precision' , 'Recall','FScore'])


result,result_labels,weighted,weighted_labels,blood,blood_labels,ask,ask_labels=prepareDatasetNoSmote(label,asklabel)
res_acc,res_scores=SVM(result,result_labels,"Result")
wei_acc,wei_scores=SVM(weighted,weighted_labels,"Weighted")
blood_acc,blood_scores=SVM(blood,blood_labels,"Blood")
ask_acc,ask_scores=SVM(ask,ask_labels,"Ask")
print("Res: Accuracy: " + str(res_acc)+" # "+ "Scores: "+str(res_scores))
print("Wei: Accuracy: " + str(wei_acc)+" # "+ "Scores: "+str(wei_scores))
print("Bld: Accuracy: " + str(blood_acc)+" # "+ "Scores: "+str(blood_scores))
print("Ask: Accuracy: " + str(ask_acc)+" # "+ "Scores: "+str(ask_scores))
Stats=Stats.append({'Dataset':"Complete",'Accuracy':res_acc,'Precision':res_scores[0],'Recall':res_scores[1],'FScore':res_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"Weighted",'Accuracy':wei_acc,'Precision':wei_scores[0],'Recall':wei_scores[1],'FScore':wei_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"Blood",'Accuracy':blood_acc,'Precision':blood_scores[0],'Recall':blood_scores[1],'FScore':blood_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"AskAPatient",'Accuracy':ask_acc,'Precision':ask_scores[0],'Recall':ask_scores[1],'FScore':ask_scores[2]},ignore_index=True)
Stats.to_excel('SVMresult_Stats'+label+'smoteless.xlsx') 


Counter({'Atenolol': 620, 'Lisinopril': 428, 'Amlodipine': 161, 'Hydrochlorothiazide': 129, 'Nadolol': 119, 'Diltiazem': 100})
Counter({'Lisinopril': 51, 'Atenolol': 45, 'Amlodipine': 14, 'Hydrochlorothiazide': 13})
Counter({'Atenolol': 186, 'Lisinopril': 134, 'Amlodipine': 63, 'Hydrochlorothiazide': 51, 'Diltiazem': 26})
Counter({'Lisinopril': 622, 'Atenolol': 134})
Counter({1: 620, 4: 428, 0: 161, 3: 129, 5: 119, 2: 100})
Counter({3: 51, 1: 45, 0: 14, 2: 13})
Counter({1: 186, 4: 134, 0: 63, 3: 51, 2: 26})
Counter({1: 622, 0: 134})
Res: Accuracy: 0.4252136752136752 # Scores: (0.29880496259806605, 0.19900557302132105, 0.1690978918393249, None)
Wei: Accuracy: 0.32432432432432434 # Scores: (0.1625, 0.20833333333333331, 0.175, None)
Bld: Accuracy: 0.3115942028985507 # Scores: (0.24755266743527934, 0.2130346304259348, 0.18980392156862744, None)
Ask: Accuracy: 0.8810572687224669 # Scores: (0.9402654867256637, 0.5178571428571429, 0.5027180527383367, None)


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### SVM 10-Fold Function

In [80]:
def SVM10Fold(result,result_labels,DataSet):
    from sklearn.model_selection import KFold
    from sklearn.svm import SVC
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import classification_report
    from sklearn.metrics import f1_score
    from collections import Counter
    kf = KFold(n_splits=10)

    accuracy=[]

    scores_rf=[]
    print(Counter(result_labels))
    count = 1
    for train_index, test_index in kf.split(result):
        X_train, X_test, Y_train, Y_test = result[train_index], result[test_index], result_labels[train_index],result_labels[test_index]
        clf = SVC(gamma='auto')
        clf.fit(X_train, Y_train)
        pred=clf.predict(X_test)
        accuracy.append(accuracy_score(Y_test, pred))
        scores_rf.append(precision_recall_fscore_support(Y_test,pred, average='macro'))
        
        #print(str(count)+DataSet)
        #print(f1_score(Y_test, pred, average='micro'))
        count+=1
        #print (pred)
        #print(classification_report(Y_test,pred,labels=[0,1]))
    accuracy = np.mean(accuracy,axis=0)   
    return accuracy,average(scores_rf)

In [92]:
def SVM10FoldNoSmote(result,result_labels,DataSet):
    from sklearn.model_selection import KFold
    from sklearn.svm import SVC
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import classification_report
    from sklearn.metrics import f1_score
    from collections import Counter
    kf = KFold(n_splits=10)
    accuracy=[]

    scores_rf=[]
    print(Counter(result_labels))
    count = 1
    for train_index, test_index in kf.split(result):
        X_train, X_test, Y_train, Y_test = result.iloc[train_index], result.iloc[test_index], result_labels[train_index],result_labels[test_index]
        clf = SVC(gamma='auto')
        clf.fit(X_train, Y_train)
        pred=clf.predict(X_test)
        accuracy.append(accuracy_score(Y_test, pred))
        scores_rf.append(precision_recall_fscore_support(Y_test,pred, average='macro'))
        
        #print(str(count)+DataSet)
        #print(f1_score(Y_test, pred, average='micro'))
        count+=1
        #print (pred)
        #print(classification_report(Y_test,pred,labels=[0,1]))
    accuracy = np.mean(accuracy,axis=0)   
    return accuracy,average(scores_rf)

### SVM 10-Fold

In [114]:
Stats = pd.DataFrame(columns = ['Dataset','Accuracy', 'Precision' , 'Recall','FScore'])

label='DrugFamily'
asklabel='DrugFamily'
result,result_labels,weighted,weighted_labels,blood,blood_labels,ask,ask_labels=prepareDataset(label,asklabel)
res_acc,res_scores=SVM10Fold(result,result_labels,"Result")
wei_acc,wei_scores=SVM10Fold(weighted,weighted_labels,"Weighted")
blood_acc,blood_scores=SVM10Fold(blood,blood_labels,"Blood")
ask_acc,ask_scores=SVM10Fold(ask,ask_labels,"Ask")
print("Res: Accuracy: " + str(res_acc)+" # "+ "Scores: "+str(res_scores))
print("Wei: Accuracy: " + str(wei_acc)+" # "+ "Scores: "+str(wei_scores))
print("Bld: Accuracy: " + str(blood_acc)+" # "+ "Scores: "+str(blood_scores))
print("Ask: Accuracy: " + str(ask_acc)+" # "+ "Scores: "+str(ask_scores))
Stats=Stats.append({'Dataset':"Complete",'Accuracy':res_acc,'Precision':res_scores[0],'Recall':res_scores[1],'FScore':res_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"Weighted",'Accuracy':wei_acc,'Precision':wei_scores[0],'Recall':wei_scores[1],'FScore':wei_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"Blood",'Accuracy':blood_acc,'Precision':blood_scores[0],'Recall':blood_scores[1],'FScore':blood_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"AskAPatient",'Accuracy':ask_acc,'Precision':ask_scores[0],'Recall':ask_scores[1],'FScore':ask_scores[2]},ignore_index=True)
Stats.to_excel('SVMresult_Stats10Fold'+label+'.xlsx') 

Counter({'Beta': 739, 'Angiotensin': 428, 'Calcium': 261, 'Diuretics': 129})
Counter({'Angiotensin': 51, 'Beta': 45, 'Calcium': 14, 'Diuretics': 13})
Counter({'Beta': 186, 'Angiotensin': 134, 'Calcium': 89, 'Diuretics': 51})
Counter({'Angiotensin': 622, 'Beta': 134})
Counter({2: 739, 3: 739, 1: 739, 0: 739})


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\class

Counter({2: 51, 3: 51, 1: 51, 0: 51})
Counter({2: 186, 3: 186, 1: 186, 0: 186})


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\class

Counter({0: 622, 1: 622})


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\class

Res: Accuracy: 0.21483967017865319 # Scores: (0.2855144910156978, 0.12461280405556421, 0.13305274494547986)
Wei: Accuracy: 0.26476190476190475 # Scores: (0.23441391941391937, 0.131765873015873, 0.1353967038073406)
Bld: Accuracy: 0.3028828828828829 # Scores: (0.2829004329004329, 0.12707094576992722, 0.1539092829688987)
Ask: Accuracy: 0.7238709677419355 # Scores: (0.5038461538461538, 0.411776837652036, 0.42419609811645004)


In [115]:
Stats = pd.DataFrame(columns = ['Dataset','Accuracy', 'Precision' , 'Recall','FScore'])

label='DrugFamily'
asklabel='DrugFamily'
result,result_labels,weighted,weighted_labels,blood,blood_labels,ask,ask_labels=prepareDatasetNoSmote(label,asklabel)
res_acc,res_scores=SVM10FoldNoSmote(result,result_labels,"Result")
wei_acc,wei_scores=SVM10FoldNoSmote(weighted,weighted_labels,"Weighted")
blood_acc,blood_scores=SVM10FoldNoSmote(blood,blood_labels,"Blood")
ask_acc,ask_scores=SVM10FoldNoSmote(ask,ask_labels,"Ask")
print("Res: Accuracy: " + str(res_acc)+" # "+ "Scores: "+str(res_scores))
print("Wei: Accuracy: " + str(wei_acc)+" # "+ "Scores: "+str(wei_scores))
print("Bld: Accuracy: " + str(blood_acc)+" # "+ "Scores: "+str(blood_scores))
print("Ask: Accuracy: " + str(ask_acc)+" # "+ "Scores: "+str(ask_scores))
Stats=Stats.append({'Dataset':"Complete",'Accuracy':res_acc,'Precision':res_scores[0],'Recall':res_scores[1],'FScore':res_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"Weighted",'Accuracy':wei_acc,'Precision':wei_scores[0],'Recall':wei_scores[1],'FScore':wei_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"Blood",'Accuracy':blood_acc,'Precision':blood_scores[0],'Recall':blood_scores[1],'FScore':blood_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"AskAPatient",'Accuracy':ask_acc,'Precision':ask_scores[0],'Recall':ask_scores[1],'FScore':ask_scores[2]},ignore_index=True)
Stats.to_excel('SVMresult_Stats10Fold'+label+'smoteless.xlsx') 

Counter({'Beta': 739, 'Angiotensin': 428, 'Calcium': 261, 'Diuretics': 129})
Counter({'Angiotensin': 51, 'Beta': 45, 'Calcium': 14, 'Diuretics': 13})
Counter({'Beta': 186, 'Angiotensin': 134, 'Calcium': 89, 'Diuretics': 51})
Counter({'Angiotensin': 622, 'Beta': 134})
Counter({1: 739, 0: 428, 2: 261, 3: 129})


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\sit

Counter({0: 51, 1: 45, 2: 14, 3: 13})
Counter({1: 186, 0: 134, 2: 89, 3: 51})


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\sit

Counter({0: 622, 1: 134})
Res: Accuracy: 0.4197146401985112 # Scores: (0.26908175871299955, 0.19293910041005888, 0.186565397032537)
Wei: Accuracy: 0.191025641025641 # Scores: (0.25, 0.11041666666666668, 0.11607142857142856)
Bld: Accuracy: 0.1673913043478261 # Scores: (0.24957770270270271, 0.07421356238975046, 0.07898883045404154)
Ask: Accuracy: 0.8173859649122808 # Scores: (0.6606666666666666, 0.6980263157894736, 0.6665890400989739)


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [89]:
result,result_labels,weighted,weighted_labels,blood,blood_labels,ask,ask_labels=prepareDatasetNoSmote(label,asklabel)

Counter({'Atenolol': 620, 'Lisinopril': 428, 'Amlodipine': 161, 'Hydrochlorothiazide': 129, 'Nadolol': 119, 'Diltiazem': 100})
Counter({'Lisinopril': 51, 'Atenolol': 45, 'Amlodipine': 14, 'Hydrochlorothiazide': 13})
Counter({'Atenolol': 186, 'Lisinopril': 134, 'Amlodipine': 63, 'Hydrochlorothiazide': 51, 'Diltiazem': 26})
Counter({'Lisinopril': 622, 'Atenolol': 134})


### Naive Bayes Function

In [98]:
def NB(result,result_labels,DataSet):
    from sklearn.model_selection import KFold
    from sklearn.naive_bayes import GaussianNB
    from sklearn.metrics import accuracy_score
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report
    from sklearn.metrics import f1_score
    from collections import Counter
    
    X_train, X_test, Y_train, Y_test = train_test_split(result, result_labels, test_size=0.30, random_state=42)
    print(Counter(result_labels))
    clf = GaussianNB()
    clf.fit(X_train, Y_train)
    pred=clf.predict(X_test)
    accuracy=(accuracy_score(Y_test, pred))
    #print (accuracy)
    scores_rf=(precision_recall_fscore_support(Y_test,pred, average='macro'))
    #print(DataSet)
    #print (pred)
    #print(classification_report(Y_test,pred,labels=[0,1,2]))
        
    return accuracy,scores_rf

### Naive Bayes 

In [179]:
label='Drug'
asklabel='Drug'
Stats = pd.DataFrame(columns = ['Dataset','Accuracy', 'Precision' , 'Recall','FScore'])


result,result_labels,weighted,weighted_labels,blood,blood_labels,ask,ask_labels=prepareDataset(label,asklabel)
res_acc,res_scores=NB(result,result_labels,"Result")
wei_acc,wei_scores=NB(weighted,weighted_labels,"Weighted")
blood_acc,blood_scores=NB(blood,blood_labels,"Blood")
ask_acc,ask_scores=NB(ask,ask_labels,"Ask")
print("Res: Accuracy: " + str(res_acc)+" # "+ "Scores: "+str(res_scores))
print("Wei: Accuracy: " + str(wei_acc)+" # "+ "Scores: "+str(wei_scores))
print("Bld: Accuracy: " + str(blood_acc)+" # "+ "Scores: "+str(blood_scores))
print("Ask: Accuracy: " + str(ask_acc)+" # "+ "Scores: "+str(ask_scores))
Stats=Stats.append({'Dataset':"Complete",'Accuracy':res_acc,'Precision':res_scores[0],'Recall':res_scores[1],'FScore':res_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"Weighted",'Accuracy':wei_acc,'Precision':wei_scores[0],'Recall':wei_scores[1],'FScore':wei_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"Blood",'Accuracy':blood_acc,'Precision':blood_scores[0],'Recall':blood_scores[1],'FScore':blood_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"AskAPatient",'Accuracy':ask_acc,'Precision':ask_scores[0],'Recall':ask_scores[1],'FScore':ask_scores[2]},ignore_index=True)
Stats.to_excel('NBresult_Stats'+label+'.xlsx') 

Counter({'Atenolol': 620, 'Lisinopril': 428, 'Amlodipine': 161, 'Hydrochlorothiazide': 129, 'Nadolol': 119, 'Diltiazem': 100})
Counter({'Lisinopril': 51, 'Atenolol': 45, 'Amlodipine': 14, 'Hydrochlorothiazide': 13})
Counter({'Atenolol': 186, 'Lisinopril': 134, 'Amlodipine': 63, 'Hydrochlorothiazide': 51, 'Diltiazem': 26})
Counter({'Lisinopril': 622, 'Atenolol': 134})
Counter({0: 620, 2: 620, 3: 620, 1: 620, 4: 620, 5: 620})
Counter({0: 51, 2: 51, 1: 51, 3: 51})
Counter({0: 186, 2: 186, 3: 186, 1: 186, 4: 186})
Counter({1: 622, 0: 622})
Res: Accuracy: 0.3010752688172043 # Scores: (0.36494080455059463, 0.3070134505980881, 0.27097004159523713, None)
Wei: Accuracy: 0.43548387096774194 # Scores: (0.4188047345374932, 0.4367209383753501, 0.39249011857707505, None)
Bld: Accuracy: 0.36200716845878134 # Scores: (0.3860378249832427, 0.3499463937621832, 0.32443780155993596, None)
Ask: Accuracy: 0.7192513368983957 # Scores: (0.7384242200933432, 0.7221554659113667, 0.715127202559285, None)


In [178]:
label='Drug'
asklabel='Drug'
Stats = pd.DataFrame(columns = ['Dataset','Accuracy', 'Precision' , 'Recall','FScore'])

result,result_labels,weighted,weighted_labels,blood,blood_labels,ask,ask_labels=prepareDataset(label,asklabel)
res_feat=importance(result,result_labels)
wei_feat=importance(weighted,weighted_labels)
blood_feat=importance(blood,blood_labels)
ask_feat=importance(ask,ask_labels)
res_acc,res_scores=NB(pd.DataFrame(result)[res_feat],result_labels,"Result")
wei_acc,wei_scores=NB(pd.DataFrame(weighted)[wei_feat],weighted_labels,"Weighted")
blood_acc,blood_scores=NB(pd.DataFrame(blood)[blood_feat],blood_labels,"Blood")
ask_acc,ask_scores=NB(pd.DataFrame(ask)[ask_feat],ask_labels,"Ask")
print("Res: Accuracy: " + str(res_acc)+" # "+ "Scores: "+str(res_scores))
print("Wei: Accuracy: " + str(wei_acc)+" # "+ "Scores: "+str(wei_scores))
print("Bld: Accuracy: " + str(blood_acc)+" # "+ "Scores: "+str(blood_scores))
print("Ask: Accuracy: " + str(ask_acc)+" # "+ "Scores: "+str(ask_scores))
Stats=Stats.append({'Dataset':"Complete",'Accuracy':res_acc,'Precision':res_scores[0],'Recall':res_scores[1],'FScore':res_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"Weighted",'Accuracy':wei_acc,'Precision':wei_scores[0],'Recall':wei_scores[1],'FScore':wei_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"Blood",'Accuracy':blood_acc,'Precision':blood_scores[0],'Recall':blood_scores[1],'FScore':blood_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"AskAPatient",'Accuracy':ask_acc,'Precision':ask_scores[0],'Recall':ask_scores[1],'FScore':ask_scores[2]},ignore_index=True)
Stats.to_excel('NBresult_Stats'+label+'importance.xlsx') 

Counter({'Atenolol': 620, 'Lisinopril': 428, 'Amlodipine': 161, 'Hydrochlorothiazide': 129, 'Nadolol': 119, 'Diltiazem': 100})
Counter({'Lisinopril': 51, 'Atenolol': 45, 'Amlodipine': 14, 'Hydrochlorothiazide': 13})
Counter({'Atenolol': 186, 'Lisinopril': 134, 'Amlodipine': 63, 'Hydrochlorothiazide': 51, 'Diltiazem': 26})
Counter({'Lisinopril': 622, 'Atenolol': 134})


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when 

Counter({0: 620, 2: 620, 3: 620, 1: 620, 4: 620, 5: 620})
Counter({0: 51, 2: 51, 1: 51, 3: 51})
Counter({0: 186, 2: 186, 3: 186, 1: 186, 4: 186})
Counter({1: 622, 0: 622})
Res: Accuracy: 0.22939068100358423 # Scores: (0.22885045363106005, 0.23107586362702695, 0.22047201696265306, None)
Wei: Accuracy: 0.41935483870967744 # Scores: (0.42354153199741434, 0.4213672969187675, 0.3820578231292517, None)
Bld: Accuracy: 0.4050179211469534 # Scores: (0.381857546153494, 0.4090253411306043, 0.36449654286747984, None)
Ask: Accuracy: 0.6978609625668449 # Scores: (0.7699468085106382, 0.7032729665550882, 0.6797241483839187, None)


In [155]:
label='Drug'
asklabel='Drug'
Stats = pd.DataFrame(columns = ['Dataset','Accuracy', 'Precision' , 'Recall','FScore'])


result,result_labels,weighted,weighted_labels,blood,blood_labels,ask,ask_labels=prepareDatasetNoSmote(label,asklabel)
res_acc,res_scores=NB(result,result_labels,"Result")
wei_acc,wei_scores=NB(weighted,weighted_labels,"Weighted")
blood_acc,blood_scores=NB(blood,blood_labels,"Blood")
ask_acc,ask_scores=NB(ask,ask_labels,"Ask")
print("Res: Accuracy: " + str(res_acc)+" # "+ "Scores: "+str(res_scores))
print("Wei: Accuracy: " + str(wei_acc)+" # "+ "Scores: "+str(wei_scores))
print("Bld: Accuracy: " + str(blood_acc)+" # "+ "Scores: "+str(blood_scores))
print("Ask: Accuracy: " + str(ask_acc)+" # "+ "Scores: "+str(ask_scores))
Stats=Stats.append({'Dataset':"Complete",'Accuracy':res_acc,'Precision':res_scores[0],'Recall':res_scores[1],'FScore':res_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"Weighted",'Accuracy':wei_acc,'Precision':wei_scores[0],'Recall':wei_scores[1],'FScore':wei_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"Blood",'Accuracy':blood_acc,'Precision':blood_scores[0],'Recall':blood_scores[1],'FScore':blood_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"AskAPatient",'Accuracy':ask_acc,'Precision':ask_scores[0],'Recall':ask_scores[1],'FScore':ask_scores[2]},ignore_index=True)
Stats.to_excel('NBresult_Stats'+label+'smoteless.xlsx') 

Counter({'Atenolol': 620, 'Lisinopril': 428, 'Amlodipine': 161, 'Hydrochlorothiazide': 129, 'Nadolol': 119, 'Diltiazem': 100})
Counter({'Lisinopril': 51, 'Atenolol': 45, 'Amlodipine': 14, 'Hydrochlorothiazide': 13})
Counter({'Atenolol': 186, 'Lisinopril': 134, 'Amlodipine': 63, 'Hydrochlorothiazide': 51, 'Diltiazem': 26})
Counter({'Lisinopril': 622, 'Atenolol': 134})
Counter({1: 620, 4: 428, 0: 161, 3: 129, 5: 119, 2: 100})
Counter({3: 51, 1: 45, 0: 14, 2: 13})
Counter({1: 186, 4: 134, 0: 63, 3: 51, 2: 26})
Counter({1: 622, 0: 134})
Res: Accuracy: 0.2606837606837607 # Scores: (0.2221957119083556, 0.25274541194816785, 0.20676555591677825, None)
Wei: Accuracy: 0.24324324324324326 # Scores: (0.2579365079365079, 0.31666666666666665, 0.22435897435897434, None)
Bld: Accuracy: 0.18840579710144928 # Scores: (0.28018018018018015, 0.21605439648917907, 0.196372749570424, None)
Ask: Accuracy: 0.6431718061674009 # Scores: (0.5776761354517734, 0.6737257717157215, 0.5436676345767255, None)


### Naive Bayes 10-Fold Function

In [105]:
def NB10Fold(result,result_labels,DataSet):
    from sklearn.model_selection import KFold
    from sklearn.naive_bayes import GaussianNB
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import classification_report
    from sklearn.metrics import f1_score
    from collections import Counter
    kf = KFold(n_splits=10)

    accuracy=[]

    scores_rf=[]
    print(Counter(result_labels))
    count = 1
    for train_index, test_index in kf.split(result):
        X_train, X_test, Y_train, Y_test = result[train_index], result[test_index], result_labels[train_index],result_labels[test_index]
        clf = GaussianNB()
        clf.fit(X_train, Y_train)
        pred=clf.predict(X_test)
        accuracy.append(accuracy_score(Y_test, pred))
        scores_rf.append(precision_recall_fscore_support(Y_test,pred, average='macro'))
        
        #print(str(count)+DataSet)
        #print(f1_score(Y_test, pred, average='micro'))
        count+=1
        #print (pred)
        #print(classification_report(Y_test,pred,labels=[0,1]))
    accuracy = np.mean(accuracy,axis=0)   
    return accuracy,average(scores_rf)

In [119]:
def NB10FoldNoSmote(result,result_labels,DataSet):
    from sklearn.model_selection import KFold
    from sklearn.naive_bayes import GaussianNB
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import classification_report
    from sklearn.metrics import f1_score
    from collections import Counter
    kf = KFold(n_splits=10)

    accuracy=[]

    scores_rf=[]
    print(Counter(result_labels))
    count = 1
    for train_index, test_index in kf.split(result):
        X_train, X_test, Y_train, Y_test = result.iloc[train_index], result.iloc[test_index], result_labels[train_index],result_labels[test_index]
        clf = GaussianNB()
        clf.fit(X_train, Y_train)
        pred=clf.predict(X_test)
        accuracy.append(accuracy_score(Y_test, pred))
        scores_rf.append(precision_recall_fscore_support(Y_test,pred, average='macro'))
        
        #print(str(count)+DataSet)
        #print(f1_score(Y_test, pred, average='micro'))
        count+=1
        #print (pred)
        #print(classification_report(Y_test,pred,labels=[0,1]))
    accuracy = np.mean(accuracy,axis=0)   
    return accuracy,average(scores_rf)

### Naive Bayes 10-Fold

In [120]:
Stats = pd.DataFrame(columns = ['Dataset','Accuracy', 'Precision' , 'Recall','FScore'])

label='DrugFamily'
asklabel='DrugFamily'
result,result_labels,weighted,weighted_labels,blood,blood_labels,ask,ask_labels=prepareDataset(label,asklabel)
res_acc,res_scores=NB10Fold(result,result_labels,"Result")
wei_acc,wei_scores=NB10Fold(weighted,weighted_labels,"Weighted")
blood_acc,blood_scores=NB10Fold(blood,blood_labels,"Blood")
ask_acc,ask_scores=NB10Fold(ask,ask_labels,"Ask")
print("Res: Accuracy: " + str(res_acc)+" # "+ "Scores: "+str(res_scores))
print("Wei: Accuracy: " + str(wei_acc)+" # "+ "Scores: "+str(wei_scores))
print("Bld: Accuracy: " + str(blood_acc)+" # "+ "Scores: "+str(blood_scores))
print("Ask: Accuracy: " + str(ask_acc)+" # "+ "Scores: "+str(ask_scores))
Stats=Stats.append({'Dataset':"Complete",'Accuracy':res_acc,'Precision':res_scores[0],'Recall':res_scores[1],'FScore':res_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"Weighted",'Accuracy':wei_acc,'Precision':wei_scores[0],'Recall':wei_scores[1],'FScore':wei_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"Blood",'Accuracy':blood_acc,'Precision':blood_scores[0],'Recall':blood_scores[1],'FScore':blood_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"AskAPatient",'Accuracy':ask_acc,'Precision':ask_scores[0],'Recall':ask_scores[1],'FScore':ask_scores[2]},ignore_index=True)
Stats.to_excel('NBresult_Stats10Fold'+label+'.xlsx') 

Counter({'Beta': 739, 'Angiotensin': 428, 'Calcium': 261, 'Diuretics': 129})
Counter({'Angiotensin': 51, 'Beta': 45, 'Calcium': 14, 'Diuretics': 13})
Counter({'Beta': 186, 'Angiotensin': 134, 'Calcium': 89, 'Diuretics': 51})
Counter({'Angiotensin': 622, 'Beta': 134})
Counter({2: 739, 3: 739, 1: 739, 0: 739})
Counter({2: 51, 3: 51, 1: 51, 0: 51})
Counter({2: 186, 3: 186, 1: 186, 0: 186})
Counter({0: 622, 1: 622})
Res: Accuracy: 0.40409413650939074 # Scores: (0.3415650772643968, 0.20843487892347806, 0.21152899249549678)
Wei: Accuracy: 0.43547619047619046 # Scores: (0.387844304388422, 0.2976629072681704, 0.29369561991452675)
Bld: Accuracy: 0.4441081081081081 # Scores: (0.3134673202614379, 0.18543418170666243, 0.21307882724499763)
Ask: Accuracy: 0.7617096774193548 # Scores: (0.5028571428571429, 0.43063670015864625, 0.43395703868260743)


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\class

In [121]:
Stats = pd.DataFrame(columns = ['Dataset','Accuracy', 'Precision' , 'Recall','FScore'])

label='DrugFamily'
asklabel='DrugFamily'
result,result_labels,weighted,weighted_labels,blood,blood_labels,ask,ask_labels=prepareDatasetNoSmote(label,asklabel)
res_acc,res_scores=NB10FoldNoSmote(result,result_labels,"Result")
wei_acc,wei_scores=NB10FoldNoSmote(weighted,weighted_labels,"Weighted")
blood_acc,blood_scores=NB10FoldNoSmote(blood,blood_labels,"Blood")
ask_acc,ask_scores=NB10FoldNoSmote(ask,ask_labels,"Ask")
print("Res: Accuracy: " + str(res_acc)+" # "+ "Scores: "+str(res_scores))
print("Wei: Accuracy: " + str(wei_acc)+" # "+ "Scores: "+str(wei_scores))
print("Bld: Accuracy: " + str(blood_acc)+" # "+ "Scores: "+str(blood_scores))
print("Ask: Accuracy: " + str(ask_acc)+" # "+ "Scores: "+str(ask_scores))
Stats=Stats.append({'Dataset':"Complete",'Accuracy':res_acc,'Precision':res_scores[0],'Recall':res_scores[1],'FScore':res_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"Weighted",'Accuracy':wei_acc,'Precision':wei_scores[0],'Recall':wei_scores[1],'FScore':wei_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"Blood",'Accuracy':blood_acc,'Precision':blood_scores[0],'Recall':blood_scores[1],'FScore':blood_scores[2]},ignore_index=True)
Stats=Stats.append({'Dataset':"AskAPatient",'Accuracy':ask_acc,'Precision':ask_scores[0],'Recall':ask_scores[1],'FScore':ask_scores[2]},ignore_index=True)
Stats.to_excel('NBresult_Stats10Fold'+label+'smoteless.xlsx') 

Counter({'Beta': 739, 'Angiotensin': 428, 'Calcium': 261, 'Diuretics': 129})
Counter({'Angiotensin': 51, 'Beta': 45, 'Calcium': 14, 'Diuretics': 13})
Counter({'Beta': 186, 'Angiotensin': 134, 'Calcium': 89, 'Diuretics': 51})
Counter({'Angiotensin': 622, 'Beta': 134})
Counter({1: 739, 0: 428, 2: 261, 3: 129})


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\class

Counter({0: 51, 1: 45, 2: 14, 3: 13})
Counter({1: 186, 0: 134, 2: 89, 3: 51})
Counter({0: 622, 1: 134})


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Ahmed\Anaconda3\lib\sit

Res: Accuracy: 0.3217990074441687 # Scores: (0.2582663749086569, 0.10603604650747937, 0.1382496165311798)
Wei: Accuracy: 0.09935897435897435 # Scores: (0.13833333333333334, 0.04930555555555556, 0.04848901098901099)
Bld: Accuracy: 0.3521739130434782 # Scores: (0.26729166666666665, 0.10604454738049278, 0.13862614924410838)
Ask: Accuracy: 0.6163157894736842 # Scores: (0.5264051522248243, 0.3400456066012489, 0.39947927964110247)


C:\Users\Ahmed\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [21]:
'''from collections import Counter
from imblearn.over_sampling import SMOTENC
sm = SMOTENC(random_state=42, categorical_features=[0, 1,3])
X_res, y_res = sm.fit_resample(result, result_labels)
print(Counter(result_labels))
print(Counter(y_res))
print(X_res)
print(y_res)
print(result_labels)'''

'from collections import Counter\nfrom imblearn.over_sampling import SMOTENC\nsm = SMOTENC(random_state=42, categorical_features=[0, 1,3])\nX_res, y_res = sm.fit_resample(result, result_labels)\nprint(Counter(result_labels))\nprint(Counter(y_res))\nprint(X_res)\nprint(y_res)\nprint(result_labels)'

In [22]:
#average(wei_scores)

IndexError: invalid index to scalar variable.

In [ ]:
print(result_labels.value_counts())
print(weighted_labels.value_counts())
print(blood_labels.value_counts())
print(ask_labels.value_counts())

In [ ]:
print(result_labels.value_counts())
print(weighted_labels.value_counts())
print(blood_labels.value_counts())
print(ask_labels.value_counts())

In [ ]:
res_scores[0]

In [ ]:
wei_acc


In [ ]:
ask_acc

In [ ]:
res_scores

In [ ]:
wei_scores

In [ ]:
ask_scores